In [80]:
import os
from pathlib import Path
import re
import shutil
import sqlite3
import yaml

from logger import logger

In [81]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

In [82]:
query = '''
SELECT
    a.id
    , a.title
    , a.path
    , b.publisher
    , c.series
    , d.format
    , d.filename
FROM books a
LEFT JOIN (
    SELECT
        x.book
        , y.publisher
    FROM books_publishers_link x
    LEFT JOIN (
    	SELECT
    		id
    		, name AS publisher
    	FROM publishers
    ) y ON x.publisher = y.id
) b ON a.id = b.book
LEFT JOIN (
    SELECT
        book
        , y.series
    FROM books_series_link x
    LEFT JOIN (
    	SELECT
    		id
    		, name AS series
    	FROM series
    ) y ON x.series = y.id
) c ON a.id = c.book
LEFT JOIN (
	SELECT
		book
		, format
        , name AS filename
	FROM data
) d ON a.id = d.book
'''

In [84]:
with sqlite3.connect("metadata.db") as connection:
    cursor = connection.cursor()
    cursor.execute(query)
    tables = cursor.fetchall()

In [85]:
tables

[(2,
  '86--EIGHTY-SIX, Vol. 1 (light novel)',
  'Asato Asato/86--EIGHTY-SIX, Vol. 1 (light novel) (2)',
  'Yen Press LLC',
  '86—EIGHTY-SIX',
  'EPUB',
  '86--EIGHTY-SIX, Vol. 1 (light novel) - Asato Asato'),
 (3,
  '86--EIGHTY-SIX, Vol. 2 (light novel)',
  'Asato Asato/86--EIGHTY-SIX, Vol. 2 (light novel) (3)',
  'Yen On',
  '86—EIGHTY-SIX',
  'EPUB',
  '86--EIGHTY-SIX, Vol. 2 (light novel) - Asato Asato'),
 (4,
  '86--EIGHTY-SIX, Vol. 3 (light novel)',
  'Asato Asato/86--EIGHTY-SIX, Vol. 3 (light novel) (4)',
  'Yen On',
  '86—EIGHTY-SIX',
  'EPUB',
  '86--EIGHTY-SIX, Vol. 3 (light novel) - Asato Asato'),
 (5,
  '86--EIGHTY-SIX, Vol. 4 (light novel)',
  'Asato Asato/86--EIGHTY-SIX, Vol. 4 (light novel) (5)',
  'Yen On',
  '86—EIGHTY-SIX',
  'EPUB',
  '86--EIGHTY-SIX, Vol. 4 (light novel) - Asato Asato'),
 (6,
  '86--EIGHTY-SIX, Vol. 5 (light novel)',
  'Asato Asato/86--EIGHTY-SIX, Vol. 5 (light novel) (6)',
  'Yen Press LLC',
  '86—EIGHTY-SIX',
  'EPUB',
  '86--EIGHTY-SIX, Vol. 5 (l

In [ ]:
# Check if destination path exist
if not os.path.exists(config.get('destination root path')):
    os.makedirs(config.get('destination root path'))


In [78]:
def sanitize_filename(filename):
    # Remove or replace any characters not allowed in filenames
    return re.sub(r'[<>:"/\\|?*]', '', filename)

In [73]:
library_path = config.get('calibre library path')
destination_root = config.get('destination root path')

for book in tables:
    name = book[1]
    path = book[2]
    publisher = book[3]
    series = book[4]
    format = book[5].lower()
    filename = book[6]

    series_dir = series if series else 'one-off'
        
    original_path = os.path.join(library_path, path, f"{filename}.{format}")
    destination_path = os.path.join(destination_root, series_dir, f"{name} [{publisher}].{format}")

In [75]:
destination_path

'\\volume1\\data\\media\\books\\calibre-series\\A Tale of the Secret Saint\\A Tale of the Secret Saint (Light Novel) Vol. 6 [Seven Seas Entertainment].epub'